# The data

In this script we will compare out 2 catalogs @kovlakasHeraklionExtragalacticCatalogue2021 and [@karachentsevUPDATEDNEARBYGALAXY2013, @karachentsevSTARFORMATIONPROPERTIES2013a]

-   The data have been joined based on their position in the sky (Ra, Dec, Distance).
-   We use TOPCAT to create two joins, an inner and an outer join
-   We will use the inner join for 1-1 comparisons
-   If we see that the data are similar we can use the outer join

In [126]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
from sklearn.metrics import r2_score
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table, join, QTable
import astropy.units as u
from astropy.visualization import quantity_support

hdul = fits.open("../tables/inner_join.fits")
dt = QTable(hdul[1].data)
hdul.close()
print(dt.info())

<QTable length=475>
      name       dtype  n_bad
--------------- ------- -----
           Name   str18     0
 Coordinates.ra float64     0
Coordinates.dec float64     0
          a26_1 float64     1
            b/a float64     1
             AB float64     0
           Kmag float64     1
         f_Kmag    str1     0
            W50   int64     0
          TType   int64     0
           Tdw1    str5     0
           Tdw2    str1     0
           RVel   int64     0
            Dis float64     0
          f_Dis    str4     0
         A26_1a float64     1
            inc   int64     0
             Vm   int64     0
         AB_int float64     1
            SBB float64     1
        logKLum float64     1
         logM26 float64    98
       l_logMHI    str1     0
         logMHI float64    84
            VLG   int64     0
         Theta1 float64    21
             MD   str17     0
         Theta5 float64    21
         Thetaj float64    51
        l_mag_B    str1     0
          mag_B floa

In [127]:
#create heatmap 
tmp  = dt.to_pandas()
#keep only the numeric columns
tmp = tmp._get_numeric_data()
correlation_matrix = tmp.corr()
#keep only the correlation that are greater than 0.8 and less than -0.8

correlation_table = pd.DataFrame.from_dict({"Column":[],
                     "Row":[],
                     "Correlation":[] })

    
correlation_table = correlation_matrix.stack().reset_index()
correlation_table = correlation_table.rename(columns={"level_0":"Column", "level_1":"Row", 0:"Correlation"})
print(correlation_table.shape)

(17400, 3)


In [128]:
correlation_table_09 = correlation_table[abs(correlation_table["Correlation"]) > 0.9]
print(correlation_table_09.shape)
# if Column and Row are the same drop the row
correlation_table = correlation_table[correlation_table["Column"]!= correlation_table["Row"]]
print(correlation_table.shape)


(376, 3)
(17268, 3)


In [129]:
Tcorr = correlation_table[correlation_table["Column"] == "T"]
Tcorr


,Column,Row,Correlation
6994,T,Coordinates.ra,0.013302
6995,T,Coordinates.dec,0.099359
6996,T,a26_1,-0.065342
6997,T,b/a,-0.026371
6998,T,AB,0.008922
...,...,...,...
7121,T,MIN_SNR,-0.484549
7122,T,METAL,-0.506390
7123,T,FLAG_METAL,0.102630
7124,T,CLASS_SP,0.104440
